In [256]:
import pandas as pd
import warnings
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
warnings.filterwarnings('ignore')

<h3>Предобработка данных</h3>

In [257]:
df = pd.read_csv('data\data_preprocessed.csv')

In [258]:
df = df[df['bundle_nm'].isin(['Без подписки', 'Pro'])]
df['pro_flg'] = df['bundle_nm'].map({'Без подписки': 0, 'Pro': 1})

In [259]:
df.dropna(subset=['age', 'gender_cd'], inplace=True) # удаление пропусков age и gender_cd
df.drop_duplicates(subset=['customer_id'], inplace=True, keep='first') # дубликаты клиентов
IQR = df["state_talk_time_sec"].quantile(0.75) - df["state_talk_time_sec"].quantile(0.25)
high = df["state_talk_time_sec"].quantile(0.75) + 1.5 * IQR
df = df[df["state_talk_time_sec"] < high] # чистка длительности звонка

In [260]:
Q1 = df['age'].quantile(0.25)
Q3 = df['age'].quantile(0.75)

IQR = Q3 - Q1

upper_bound = Q3 + 1.5 * IQR

In [261]:
df_filter = df[(df['age'] >= 14) & (df['age'] <= upper_bound)]
df_filter[df_filter['product'] == 'Кредит']['product'] = df_filter[df_filter['age'] >= 18]['product']
df_filter[df_filter['product'] == 'Кредитная карта']['product'] = df_filter[df_filter['age'] >= 18]['product']

In [262]:
df_filter['age'] = scaler.fit_transform(df_filter[['age']])
df_filter['state_talk_time_sec'] = scaler.fit_transform(df_filter[['state_talk_time_sec']])

In [263]:
df_filter.to_csv('data/data_final_preprocessed.csv', index=False)